In [1]:
import requests
import json

In [4]:
# URL without pagination parameters
url = 'https://gisportal.ers.usda.gov/server/rest/services/Rural_Atlas_Data/County_Classifications/MapServer/2/query?where=1%3D1&outFields=*&outSR=4326&f=pjson'

# Initialize the pagination parameters
offset = 0
count = 1000
data = None

while True:
    # Add pagination parameters to URL
    paginated_url = f"{url}&resultOffset={offset}&resultRecordCount={count}"
    response = requests.get(paginated_url)
    page_data = response.json()

    # On the first page, initialize the data with the page data
    if data is None:
        data = page_data
    else:
        # On subsequent pages, add the features from the page data to the main data
        data['features'].extend(page_data['features'])

    # If the number of features in the page is less than the count, this is the last page
    if len(page_data['features']) < count:
        break

    # Increase the offset for the next page
    offset += count

# Save the data
with open('metroareas.json', 'w') as f:
    json.dump(data, f)

In [8]:
import json

# Load the original data
with open('metroareas.json', 'r') as f:
    data = json.load(f)

# Define the structure of the new data
new_data = {
    'type': 'FeatureCollection',
    'features': []
}

# Transform the original data into the new structure
for feature in data['features']:
    new_feature = {
        'type': 'Feature',
        'properties': feature['attributes'],
        'geometry': {
            'type': 'Polygon',
            'coordinates': feature['geometry']['rings']
        }
    }
    new_data['features'].append(new_feature)

# Save the new data
with open('new_metrodata.geojson', 'w') as f:
    json.dump(new_data, f)
